In [24]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline 

from skimage import io, color, filters, feature, restoration
from skimage.transform import resize, rotate
from skimage.color import rgb2gray
from scipy.spatial.distance import squareform
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score
from scipy.misc import imread
import pandas as pd
import matplotlib.cm as cm
from scipy import ndimage, misc
import pickle

In [2]:
import sys
sys.path.append("..")
from src.classification import Classifiers
from src.model_prep import prepareDF


/Users/reiffd/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
line_imgs = glob.glob('../../line_remover_2_data_stash/gray/all/lines/*')
drawing_imgs = glob.glob('../../line_remover_2_data_stash/gray/all/drawings/*')

In [4]:
len(drawing_imgs)

5682

In [5]:
vectorized = np.zeros((5691, 900))

In [6]:
first = imread(line_imgs[0], mode='L').flatten()

/Users/reiffd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


In [44]:
model.predict(first.reshape(1, -1))[0]

1

In [7]:
vectorized[0, :] = first

In [8]:
def vectorize_imgs(img_list, n):
    result = np.zeros((n, 900))
    for i, img in enumerate(img_list):
        result[i, :] = imread(img, mode='L').flatten()
    return result
    

In [9]:
lines_vec = vectorize_imgs(line_imgs, 5691)

/Users/reiffd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


In [10]:
drawings_vec = vectorize_imgs(drawing_imgs, 5682)

/Users/reiffd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


In [11]:
lines_df = pd.DataFrame(lines_vec)
drawings_df = pd.DataFrame(drawings_vec)

In [12]:
lines_df['label'] = 1
drawings_df['label'] = 0

In [13]:
drawings_df

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,label
0,199.0,192.0,188.0,174.0,179.0,177.0,179.0,190.0,197.0,214.0,...,248.0,250.0,250.0,248.0,248.0,248.0,246.0,245.0,248.0,0
1,48.0,46.0,44.0,50.0,46.0,52.0,36.0,31.0,46.0,24.0,...,231.0,229.0,231.0,231.0,239.0,235.0,239.0,231.0,235.0,0
2,107.0,190.0,238.0,243.0,248.0,206.0,113.0,235.0,224.0,152.0,...,55.0,79.0,130.0,121.0,46.0,73.0,147.0,159.0,211.0,0
3,241.0,243.0,241.0,237.0,241.0,237.0,239.0,237.0,234.0,237.0,...,241.0,243.0,245.0,246.0,245.0,241.0,245.0,245.0,248.0,0
4,232.0,232.0,226.0,226.0,240.0,226.0,220.0,227.0,187.0,175.0,...,243.0,240.0,246.0,246.0,243.0,236.0,243.0,234.0,224.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677,68.0,68.0,78.0,92.0,147.0,201.0,217.0,229.0,239.0,239.0,...,243.0,245.0,243.0,243.0,253.0,245.0,243.0,241.0,249.0,0
5678,92.0,96.0,77.0,60.0,81.0,89.0,77.0,60.0,68.0,104.0,...,249.0,242.0,247.0,247.0,243.0,242.0,247.0,245.0,245.0,0
5679,240.0,240.0,242.0,243.0,234.0,224.0,222.0,222.0,236.0,227.0,...,238.0,243.0,248.0,242.0,243.0,238.0,238.0,238.0,242.0,0
5680,245.0,241.0,241.0,241.0,241.0,245.0,245.0,237.0,241.0,237.0,...,245.0,241.0,245.0,245.0,241.0,237.0,237.0,241.0,245.0,0


In [14]:
lines_df

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,label
0,242.0,239.0,239.0,239.0,237.0,245.0,204.0,215.0,245.0,239.0,...,247.0,245.0,242.0,242.0,242.0,242.0,245.0,242.0,245.0,1
1,246.0,243.0,243.0,246.0,243.0,238.0,230.0,147.0,87.0,217.0,...,242.0,243.0,242.0,242.0,248.0,246.0,246.0,246.0,243.0,1
2,243.0,239.0,241.0,237.0,241.0,245.0,241.0,235.0,239.0,245.0,...,239.0,241.0,245.0,241.0,241.0,243.0,243.0,243.0,247.0,1
3,246.0,246.0,242.0,240.0,248.0,188.0,157.0,240.0,240.0,242.0,...,89.0,93.0,192.0,129.0,250.0,238.0,242.0,240.0,238.0,1
4,242.0,240.0,243.0,230.0,46.0,109.0,56.0,42.0,105.0,173.0,...,242.0,242.0,240.0,240.0,242.0,242.0,243.0,243.0,242.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,215.0,245.0,245.0,235.0,225.0,215.0,225.0,235.0,225.0,215.0,...,235.0,235.0,225.0,235.0,245.0,235.0,215.0,215.0,225.0,1
5687,235.0,231.0,231.0,231.0,229.0,226.0,226.0,226.0,231.0,231.0,...,130.0,162.0,143.0,0.0,99.0,113.0,72.0,67.0,22.0,1
5688,213.0,234.0,224.0,234.0,213.0,213.0,224.0,213.0,202.0,192.0,...,224.0,234.0,224.0,213.0,213.0,234.0,192.0,181.0,224.0,1
5689,209.0,218.0,230.0,226.0,235.0,226.0,218.0,214.0,209.0,230.0,...,222.0,230.0,222.0,222.0,218.0,205.0,239.0,235.0,222.0,1


In [15]:
dfs = [drawings_df, lines_df]
df = pd.concat(dfs, axis=0, ignore_index = 0)

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,label
0,199.0,192.0,188.0,174.0,179.0,177.0,179.0,190.0,197.0,214.0,...,248.0,250.0,250.0,248.0,248.0,248.0,246.0,245.0,248.0,0
1,48.0,46.0,44.0,50.0,46.0,52.0,36.0,31.0,46.0,24.0,...,231.0,229.0,231.0,231.0,239.0,235.0,239.0,231.0,235.0,0
2,107.0,190.0,238.0,243.0,248.0,206.0,113.0,235.0,224.0,152.0,...,55.0,79.0,130.0,121.0,46.0,73.0,147.0,159.0,211.0,0
3,241.0,243.0,241.0,237.0,241.0,237.0,239.0,237.0,234.0,237.0,...,241.0,243.0,245.0,246.0,245.0,241.0,245.0,245.0,248.0,0
4,232.0,232.0,226.0,226.0,240.0,226.0,220.0,227.0,187.0,175.0,...,243.0,240.0,246.0,246.0,243.0,236.0,243.0,234.0,224.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,215.0,245.0,245.0,235.0,225.0,215.0,225.0,235.0,225.0,215.0,...,235.0,235.0,225.0,235.0,245.0,235.0,215.0,215.0,225.0,1
5687,235.0,231.0,231.0,231.0,229.0,226.0,226.0,226.0,231.0,231.0,...,130.0,162.0,143.0,0.0,99.0,113.0,72.0,67.0,22.0,1
5688,213.0,234.0,224.0,234.0,213.0,213.0,224.0,213.0,202.0,192.0,...,224.0,234.0,224.0,213.0,213.0,234.0,192.0,181.0,224.0,1
5689,209.0,218.0,230.0,226.0,235.0,226.0,218.0,214.0,209.0,230.0,...,222.0,230.0,222.0,222.0,218.0,205.0,239.0,235.0,222.0,1


In [17]:
prepare = prepareDF(df, True)

In [18]:
X_train, X_test, y_train, y_test = prepare.X_train, prepare.X_test, prepare.y_train, prepare.y_test

In [20]:
param_dist = {"booster": ["gbtree", "gblinear"],
               "eta": np.linspace(0, 1, 9),
               "subsample": np.linspace(0.5, 1, 5),
               "max_depth": np.arange(3, 10),
               "colsample_bytree": np.linspace(0.5, 1, 5),
               "lambda": np.linspace(0, 1, 9),
               "alpha": np.linspace(0, 1, 9),
               "objective": ['binary:logistic'],
               "eval_metric": ['rmse', 'mae', 'error']}

In [21]:
rf_model = Classifiers(RandomForestClassifier(), param_dist)

In [22]:
rf_model.fit(X_train, y_train)

In [23]:
rf_model.predict(X_test)

In [25]:
print("Accuracy: {}, Precision: {}, Recall: {}".format(accuracy_score(y_test, rf_model.y_pred), precision_score(y_test, rf_model.y_pred), recall_score(y_test, rf_model.y_pred)))

Accuracy: 0.9609704641350211, Precision: 0.9887133182844243, Recall: 0.9319148936170213


In [26]:
pickle.dump(rf_model.model, open('../models/models/rf_model_flat.sav', 'wb'))

In [27]:
model = pickle.load(open('../models/models/rf_model_flat.sav', 'rb'))

In [36]:
x = df.drop(columns=['label']).iloc[0, :].to_numpy().reshape(1, -1)

In [38]:
model.predict(x)[0]

0